# Pipeline de entrenamiento y predicción

En este notebook se ejemplifica como ejecutar los scripts para poder:
    
1. Entrenar un modelo para clasificar las zonas de un mapa
2. Usar ese mismo modelo para analizar zonas y tener un estimativo de las zonas 

## Preparando los datos

En este notebook se demostrará el proceso utilizando un recorte de imágenes SPOT6/7 para la ciudad de Concordia, Entre Ríos. Se utilizará la imagen correspondiente al año 2017 para entrenar el modelo, y la del 2018 para evaluarlo.

In [1]:
import os

root_path = os.getcwd()

images_path = os.path.join(root_path, 'data', 'imagenes')
train_tif_path = os.path.join(images_path, 'concordia_2016', 'concordia_small_2016.tif')
test_tif_path = os.path.join(images_path, 'concordia_2018', 'concordia_small_2018.tif')

*Opcional*: Si quiere ejecutar el proceso para las imagenes enteras, descomente las lineas de la siguiente celda y ejecútela.

**Tenga en cuenta que el entrenamiento puede demorar varias horas.**

In [2]:
def download_file(url, out_path):
    if os.path.exists(out_path):
        print("El archivo {} ya existe. Si quiere descargarlo de nuevo, debe borrarlo primero.".format(out_path))
        return

    #!wget -O $test_tif_path $test_remote_tif_url
    r = requests.get(url, stream=True) 
    with open(out_path, "wb") as f: 
        i = 0
        for chunk in r.iter_content(chunk_size=available_mem): 
            # writing one chunk at a time
            if chunk: 
                f.write(chunk)
                clear_output(wait=True)
                print("Escribiendo bloque {} de test, de {} bytes".format(i,available_mem))
                i += 1

In [3]:
# concordia_data_path = '{}/data/conae/centro/concordia'.format(root_path)

# train_product_folder = 'train'
# train_product_image = 'concordia_2016.TIF'
# train_tif_path = '{}/{}/{}'.format(concordia_data_path,train_product_folder,train_product_image)
# train_remote_tif_url = 'https://storage.googleapis.com/dym-ap-siu-habitat-public-images/concordia/IMG_SPOT7_PMS_201601281334024_ORT_C0000000058530_R1C1.TIF'

# !wget --continue -O $train_tif_path $train_remote_tif_url

# test_product_folder = 'test'
# test_product_image = 'concordia_2018.TIF'
# test_tif_path = '{}/{}/{}'.format(concordia_data_path,test_product_folder,test_product_image)
# test_remote_tif_url = 'https://storage.googleapis.com/dym-ap-siu-habitat-public-images/concordia/IMG_SPOT6_PMS_201801071323410_ORT_C0000000058510_R1C1.TIF'

# !wget --continue -O $test_tif_path $test_remote_tif_url

Inicializamos el resto de las variables para ejecutar los scripts, entre ellas la cantidad de hilos paralelos (`n_jobs`) , arboles (`trees`) y profundidad de los mismos (`depth`). Además indicamos las rutas de los scripts, archivos vectoriales etiquetados y rutas de salida.

In [4]:
script_train_path = '{}/script/siu_train.py'.format(root_path)
script_test_path = '{}/script/siu_test.py'.format(root_path)

n_jobs = 4
trees = 107
depth = 1007

train_shp_path = '{}/data/etiquetado/centro/concordia2016_small_train.shp'.format(root_path)
out_train_model_path = '{}/data/concordia2016_train_rf_t{}_d{}'.format(root_path,trees,depth)
model_file_name = 'concordia2016_train_rf_t{trs}_d{dpt}_rf_t{trs}_d{dpt}.txt'.format(trs=trees,dpt=depth)

test_shp_path = '{}/data/etiquetado/centro/concordia2018_small_train.shp'.format(root_path)
out_test_results_path = '{}/data/concordia2018_train_rf_t{}_d{}'.format(root_path,trees,depth)

out_train_model_path_full = '{}/model_train_rf_t{}_d{}/{}'.format(out_train_model_path,trees,depth,model_file_name)
out_test_results_path_train = '{}/data/concordia2016_test_rf_t{}_d{}'.format(root_path,trees,depth)

## Entrenamiento del modelo

Entrenamos a modo de ejemplo sobre los datos de la ciudad de concordia del año 2016

In [5]:
# Para ejecutarlo en el notebook, descomentar y usar la siguente linea 
!python3 $script_train_path $train_tif_path $train_shp_path $out_train_model_path $n_jobs $trees $depth --root_path $root_path

# Para ejecutarlo en background independiente del notbook usar la siguente linea
#!python3 $script_train_path $train_tif_path $train_shp_path $out_train_model_path $n_jobs $trees $depth --root_path $root_path >> ./concordia2016_train_rf.log 2>&1

=>    1. Generando features
02:04:55:INFO:409:spfeas.main:
Start date & time --- (Mon Nov 25 02:04:55 2019)

02:04:55:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 4 of 6 ...
02:04:55:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:04:55:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
02:04:55:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 3 of 6 ...
02:04:55:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 1 of 6 ...
02:04:55:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 2 of 6 ...
02:04:58:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:04:58:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
02:05:02:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:05:02:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
02:05:02:INFO:28:spreshape.r

02:05:41:INFO:628:spprocess.run:  Updating status ...
02:05:41:INFO:628:spprocess.run:  Updating status ...
02:05:41:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 4 of 6 ...
02:05:41:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:05:41:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
02:05:46:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 3 of 6 ...
02:05:46:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:05:46:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
02:05:51:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 2 of 6 ...
02:05:51:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:05:51:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
02:05:51:INFO:512:spsplit.get_section_stats:  Processing Differential Mor

02:06:35:INFO:628:spprocess.run:  Updating status ...
02:06:36:INFO:628:spprocess.run:  Updating status ...
02:06:36:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 4 of 6 ...
02:06:36:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:06:36:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
02:06:36:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 2 of 6 ...
02:06:36:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 3 of 6 ...
02:06:36:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 1 of 6 ...
02:06:40:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:06:40:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
02:06:42:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:06:42:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
02:06:44:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...


02:07:06:INFO:628:spprocess.run:  Updating status ...
02:07:07:INFO:628:spprocess.run:  Updating status ...
02:07:07:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 4 of 6 ...
02:07:07:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:07:07:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
02:07:07:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 3 of 6 ...
02:07:07:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 1 of 6 ...
02:07:07:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 2 of 6 ...
02:07:08:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:07:08:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
02:07:10:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:07:10:INFO:67:spprocess._write_section2file:  Writing section 

02:07:21:INFO:628:spprocess.run:  Updating status ...
02:07:21:INFO:628:spprocess.run:  Updating status ...
02:07:21:INFO:512:spsplit.get_section_stats:  Processing Histogram of Oriented Gradients for section 4 of 6 ...
02:07:21:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:07:21:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
02:07:21:INFO:512:spsplit.get_section_stats:  Processing Histogram of Oriented Gradients for section 3 of 6 ...
02:07:21:INFO:512:spsplit.get_section_stats:  Processing Histogram of Oriented Gradients for section 1 of 6 ...
02:07:21:INFO:512:spsplit.get_section_stats:  Processing Histogram of Oriented Gradients for section 2 of 6 ...
02:07:24:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:07:24:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
02:07:27:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:07:27:INFO:67:spprocess._write_section2file:  Writing 

02:07:54:INFO:628:spprocess.run:  Updating status ...
02:07:54:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 5 of 6 ...
02:07:54:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:07:54:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
02:07:54:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 6 of 6 ...
02:07:54:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:07:54:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
02:07:54:INFO:628:spprocess.run:  Updating status ...
02:07:54:INFO:628:spprocess.run:  Updating status ...
02:07:55:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 4 of 6 ...
02:07:55:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:07:55:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
02:07:55:INFO:512:spsplit.get_section_stats:  Processing Lacunarity for section 1 of 6 ...
02:

02:08:03:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:08:03:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
02:08:03:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:08:03:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
02:08:04:INFO:628:spprocess.run:  Updating status ...
02:08:04:INFO:512:spsplit.get_section_stats:  Processing Normalized Difference Vegetation Index for section 6 of 6 ...
02:08:04:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:08:04:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
02:08:04:INFO:512:spsplit.get_section_stats:  Processing Normalized Difference Vegetation Index for section 5 of 6 ...
02:08:04:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:08:04:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
02:08:04:INFO:628:spprocess.run:  Updating status ...
02:08:04:INFO:628:spproces

02:08:14:INFO:628:spprocess.run:  Updating status ...
02:08:14:INFO:512:spsplit.get_section_stats:  Processing Mean for section 5 of 6 ...
02:08:14:INFO:512:spsplit.get_section_stats:  Processing Mean for section 6 of 6 ...
02:08:14:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:08:14:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
02:08:14:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:08:14:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
02:08:14:INFO:628:spprocess.run:  Updating status ...
02:08:14:INFO:628:spprocess.run:  Updating status ...
02:08:14:INFO:512:spsplit.get_section_stats:  Processing Mean for section 4 of 6 ...
02:08:15:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:08:15:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
02:08:15:INFO:512:spsplit.get_section_stats:  Processing Mean for section 1 of 6 ...
02:08:15:INFO:512:spsplit.g

02:08:35:INFO:58:sample_raster._sample_parallel:Band 79 of 288 ...
02:08:35:INFO:58:sample_raster._sample_parallel:Band 80 of 288 ...
02:08:35:INFO:58:sample_raster._sample_parallel:Band 81 of 288 ...
02:08:35:INFO:58:sample_raster._sample_parallel:Band 82 of 288 ...
02:08:36:INFO:58:sample_raster._sample_parallel:Band 83 of 288 ...
02:08:36:INFO:58:sample_raster._sample_parallel:Band 84 of 288 ...
02:08:36:INFO:58:sample_raster._sample_parallel:Band 85 of 288 ...
02:08:36:INFO:58:sample_raster._sample_parallel:Band 86 of 288 ...
02:08:36:INFO:58:sample_raster._sample_parallel:Band 87 of 288 ...
02:08:36:INFO:58:sample_raster._sample_parallel:Band 88 of 288 ...
02:08:36:INFO:58:sample_raster._sample_parallel:Band 89 of 288 ...
02:08:36:INFO:58:sample_raster._sample_parallel:Band 90 of 288 ...
02:08:36:INFO:58:sample_raster._sample_parallel:Band 91 of 288 ...
02:08:36:INFO:58:sample_raster._sample_parallel:Band 92 of 288 ...
02:08:36:INFO:58:sample_raster._sample_parallel:Band 93 of 288

02:08:44:INFO:58:sample_raster._sample_parallel:Band 200 of 288 ...
02:08:44:INFO:58:sample_raster._sample_parallel:Band 201 of 288 ...
02:08:44:INFO:58:sample_raster._sample_parallel:Band 202 of 288 ...
02:08:44:INFO:58:sample_raster._sample_parallel:Band 203 of 288 ...
02:08:44:INFO:58:sample_raster._sample_parallel:Band 204 of 288 ...
02:08:44:INFO:58:sample_raster._sample_parallel:Band 205 of 288 ...
02:08:44:INFO:58:sample_raster._sample_parallel:Band 206 of 288 ...
02:08:44:INFO:58:sample_raster._sample_parallel:Band 207 of 288 ...
02:08:44:INFO:58:sample_raster._sample_parallel:Band 208 of 288 ...
02:08:45:INFO:58:sample_raster._sample_parallel:Band 209 of 288 ...
02:08:45:INFO:58:sample_raster._sample_parallel:Band 210 of 288 ...
02:08:45:INFO:58:sample_raster._sample_parallel:Band 211 of 288 ...
02:08:45:INFO:58:sample_raster._sample_parallel:Band 212 of 288 ...
02:08:45:INFO:58:sample_raster._sample_parallel:Band 213 of 288 ...
02:08:45:INFO:58:sample_raster._sample_parallel:

In [6]:
print("Entrenamiento terminado")

Entrenamiento terminado


Podemos analizar la carpeta de salida donde encontramos el resultado de performance y el modelo obtenido

In [7]:
out_folder = '{}/model_train_rf_t{}_d{}/'.format(out_train_model_path,trees,depth)
# !ls -lth $out_folder
os.listdir(out_folder)

['concordia2016_train_rf_t107_d1007_rf_t107_d1007_acc.txt',
 'concordia2016_train_rf_t107_d1007_rf_t107_d1007.txt']

Abriendo el archivo terminado en `_acc.txt` vemos la matriz de confusión y las medidas de performance de entrenamiento

In [8]:
performance_file = '{}{}'.format(out_folder,'concordia2016_train_rf_t107_d1007_rf_t107_d1007_acc.txt')
#!cat $performance_file
with open(performance_file, 'r') as f:
    print(f.read())

Error Matrix

                Observed
                --------
                C     1   C     2   C     3   C     4   C     5   Total   User(%)
                -------   -------   -------   -------   -------   -----   -------
Predicted C001| (25)      3         1         1         0         30      83.33
          C002| 9         (100)     0         2         0         111     90.09
          C003| 0         0         (131)     0         0         131     100.00
          C004| 0         0         1         (125)     0         126     99.21
          C005| 0         0         0         0         (498)     498     100.00
         Total| 34        103       133       128       498       (896)
   Producer(%)| 73.53     97.09     98.50     97.66     100.00            (98.10%)

Samples: 896

Statistics

Overall Accuracy (%): 98.10
Kappa: 0.97
Hamming loss: 0.02

Class report

             precision    recall  f1-score   support

          1       0.83      0.74      0.78        34
       

## Predicción usando el modelo aprendido

Usamos el modelo aprendido para analizar la misma zona pero del año 2018


In [9]:
# Para ejecutarlo en el notebook, descomentar y usar la siguente linea 
!python3 $script_test_path $test_tif_path $test_shp_path $out_train_model_path_full $out_test_results_path $n_jobs $trees $depth --root_path $root_path
!python3 $script_test_path $train_tif_path $train_shp_path $out_train_model_path_full $out_test_results_path_train $n_jobs $trees $depth --root_path $root_path

# Para ejecutarlo en background independiente del notbook usar la siguente linea
# Sobre el test set (2018)
#!python3 $script_test_path $test_tif_path $test_shp_path $out_train_model_path_full $out_test_results_path $n_jobs $trees $depth --root_path $root_path >> ./concordia2018_test_rf.log 2>&1
# Sobre el train set (2016)
#!python3 $script_test_path $train_tif_path $train_shp_path $out_train_model_path_full $out_test_results_path_train $n_jobs $trees $depth --root_path $root_path >> ./concordia2016_test_train_rf.log 2>&1

=>    1. Generando features
02:09:08:INFO:409:spfeas.main:
Start date & time --- (Mon Nov 25 02:09:08 2019)

02:09:08:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 4 of 6 ...
02:09:08:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:09:08:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
02:09:08:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 3 of 6 ...
02:09:08:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 1 of 6 ...
02:09:08:INFO:512:spsplit.get_section_stats:  Processing Fourier transfrom for section 2 of 6 ...
02:09:14:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:09:14:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
02:09:15:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:09:15:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
02:09:19:INFO:28:spreshape.r

02:09:57:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 5 of 6 ...
02:09:57:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:09:57:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
02:09:57:INFO:628:spprocess.run:  Updating status ...
02:09:57:INFO:628:spprocess.run:  Updating status ...
02:09:58:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 4 of 6 ...
02:09:58:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:09:58:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
02:10:01:INFO:512:spsplit.get_section_stats:  Processing Differential Morphological Profiles for section 3 of 6 ...
02:10:01:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:10:01:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
02:10:07:INFO:512:spsplit.get_section_stats:  Processing Differential Mor

02:10:51:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:10:51:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
02:10:52:INFO:628:spprocess.run:  Updating status ...
02:10:52:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 6 of 6 ...
02:10:52:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 5 of 6 ...
02:10:52:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:10:52:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:10:52:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
02:10:52:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
02:10:52:INFO:628:spprocess.run:  Updating status ...
02:10:52:INFO:628:spprocess.run:  Updating status ...
02:10:52:INFO:512:spsplit.get_section_stats:  Processing PanTex for section 4 of 6 ...
02:10:52:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:10:52:INFO:67:spprocess._write

02:11:21:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:11:21:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
02:11:21:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:11:21:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
02:11:22:INFO:628:spprocess.run:  Updating status ...
02:11:22:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 6 of 6 ...
02:11:22:INFO:512:spsplit.get_section_stats:  Processing Local Binary Patterns moments for section 5 of 6 ...
02:11:22:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:11:22:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
02:11:22:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:11:22:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
02:11:23:INFO:628:spprocess.run:  Updating status ...
02:11:23:INFO:628:spprocess.run:  Updating s

02:11:35:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:11:35:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
02:11:35:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:11:35:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
02:11:36:INFO:512:spsplit.get_section_stats:  Processing Gabor filters for section 1 of 6 ...
02:11:36:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:11:36:INFO:67:spprocess._write_section2file:  Writing section 1 of 6 to file ...
02:11:37:INFO:628:spprocess.run:  Updating status ...
02:11:37:INFO:512:spsplit.get_section_stats:  Processing Gabor filters for section 6 of 6 ...
02:11:37:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:11:37:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
02:11:37:INFO:512:spsplit.get_section_stats:  Processing Gabor filters for section 5 of 6 ...
02:11:37:INFO:28:spreshape.reshape_f

...

02:16:39:INFO:628:spprocess.run:  Updating status ...
02:16:39:INFO:512:spsplit.get_section_stats:  Processing Normalized Difference Vegetation Index for section 5 of 6 ...
02:16:39:INFO:512:spsplit.get_section_stats:  Processing Normalized Difference Vegetation Index for section 6 of 6 ...
02:16:39:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:16:39:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
02:16:39:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:16:39:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
02:16:39:INFO:628:spprocess.run:  Updating status ...
02:16:40:INFO:628:spprocess.run:  Updating status ...
02:16:40:INFO:512:spsplit.get_section_stats:  Processing Normalized Difference Vegetation Index for section 4 of 6 ...
02:16:40:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:16:40:INFO:67:spprocess._write_section2file:  Writing section 4 of 6 to file ...
02:16:40:I

02:16:48:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:16:48:INFO:67:spprocess._write_section2file:  Writing section 3 of 6 to file ...
02:16:48:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:16:48:INFO:67:spprocess._write_section2file:  Writing section 2 of 6 to file ...
02:16:49:INFO:628:spprocess.run:  Updating status ...
02:16:49:INFO:512:spsplit.get_section_stats:  Processing Mean for section 6 of 6 ...
02:16:49:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:16:49:INFO:67:spprocess._write_section2file:  Writing section 6 of 6 to file ...
02:16:49:INFO:512:spsplit.get_section_stats:  Processing Mean for section 5 of 6 ...
02:16:49:INFO:28:spreshape.reshape_feature_list:  Reshaping features ...
02:16:49:INFO:67:spprocess._write_section2file:  Writing section 5 of 6 to file ...
02:16:49:INFO:628:spprocess.run:  Updating status ...
02:16:50:INFO:628:spprocess.run:  Updating status ...
02:16:50:INFO:512:spsplit.get_section_st

02:17:07:INFO:58:sample_raster._sample_parallel:Band 54 of 288 ...
02:17:07:INFO:58:sample_raster._sample_parallel:Band 55 of 288 ...
02:17:07:INFO:58:sample_raster._sample_parallel:Band 56 of 288 ...
02:17:07:INFO:58:sample_raster._sample_parallel:Band 57 of 288 ...
02:17:07:INFO:58:sample_raster._sample_parallel:Band 58 of 288 ...
02:17:07:INFO:58:sample_raster._sample_parallel:Band 59 of 288 ...
02:17:07:INFO:58:sample_raster._sample_parallel:Band 60 of 288 ...
02:17:07:INFO:58:sample_raster._sample_parallel:Band 61 of 288 ...
02:17:07:INFO:58:sample_raster._sample_parallel:Band 62 of 288 ...
02:17:08:INFO:58:sample_raster._sample_parallel:Band 63 of 288 ...
02:17:08:INFO:58:sample_raster._sample_parallel:Band 64 of 288 ...
02:17:08:INFO:58:sample_raster._sample_parallel:Band 65 of 288 ...
02:17:08:INFO:58:sample_raster._sample_parallel:Band 66 of 288 ...
02:17:08:INFO:58:sample_raster._sample_parallel:Band 67 of 288 ...
02:17:08:INFO:58:sample_raster._sample_parallel:Band 68 of 288

02:17:15:INFO:58:sample_raster._sample_parallel:Band 176 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 177 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 178 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 179 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 180 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 181 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 182 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 183 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 184 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 185 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 186 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 187 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 188 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:Band 189 of 288 ...
02:17:16:INFO:58:sample_raster._sample_parallel:

02:17:29:INFO:146:classify.main:
Start date & time --- (Mon Nov 25 02:17:29 2019)

02:17:29:INFO:3962:classification._load_model:  Loading /ap-siu-habitat/data/concordia2016_train_rf_t107_d1007/model_train_rf_t107_d1007/concordia2016_train_rf_t107_d1007_rf_t107_d1007.txt ...
02:17:30:INFO:6639:classification.test_accuracy:  Getting test accuracy ...
02:17:30:INFO:5520:classification.predict:  Predicting class labels from /ap-siu-habitat/data/concordia2016_test_rf_t107_d1007/features_test/concordia_small_2016__BD1-2-3-4_BK8_SC8-16_TRfourier-dmp-pantex-lbpm-gabor-hog-lac-ndvi-mean.vrt and writing to /ap-siu-habitat/data/concordia2016_test_rf_t107_d1007/clasificacion_test_rf_t107_d1007/concordia2016_test_rf_t107_d1007_rf_t107_d1007.tif ...
02:17:30:INFO:5775:classification._predict:  Block 1 of 1 ...
02:17:34:INFO:197:classify.main:
End data & time -- (Mon Nov 25 02:17:34 2019)
Total processing time -- (5s)



In [10]:
print("Evaluacion terminada")

Evaluacion terminada


La performance sobre el año 2018 usado de test es la siguiente

In [11]:
out_folder = '{}/model_test_rf_t{}_d{}/'.format(out_test_results_path,trees,depth)
performance_file = '{}{}'.format(out_folder, 'concordia2018_train_rf_t107_d1007_rf_t107_d1007_acc.txt')
# !cat $performance_file
with open(performance_file, 'r') as f:
    print(f.read())

Error Matrix

                Observed
                --------
                C     1   C     2   C     3   C     4   C     5   Total   User(%)
                -------   -------   -------   -------   -------   -----   -------
Predicted C001| (22)      7         135       3         0         167     13.17
          C002| 10        (85)      0         10        0         105     80.95
          C003| 1         1         (21)      1         0         24      87.50
          C004| 1         4         2         (46)      2         55      83.64
          C005| 0         0         0         0         (844)     844     100.00
         Total| 34        97        158       60        846       (1195)
   Producer(%)| 64.71     87.63     13.29     76.67     99.76             (85.19%)

Samples: 1,195

Statistics

Overall Accuracy (%): 85.19
Kappa: 0.69
Hamming loss: 0.15

Class report

             precision    recall  f1-score   support

          1       0.13      0.65      0.22        34
     

Vemos que en la carpeta `clasificacion_...` dentro de la carpeta de salida, está la imagen obtenida luego de aplicar el modelo

In [12]:
map_file = '{}/clasificacion_test_rf_t{}_d{}/concordia2018_train_rf_t107_d1007_rf_t107_d1007.tif'.format(out_test_results_path, trees, depth)
# !ls $map_file
os.listdir(os.path.dirname(map_file))

['concordia2018_train_rf_t107_d1007_rf_t107_d1007.tif']

## Mover los resultados a una carpeta con formato esperado para post-procesamiento

Movemos los resultados a una estructura de carpetas con el patrón `region/ciudad/año` que esperan los scripts de post-procesamiento 

In [22]:
import subprocess
import glob
import shutil

def prepare_results(*, year, path_salida, region_city, resultados_classifier, imgs_orig, raster_virtual_path):
    # Crear el directorio
    results_full_path = os.path.join(path_salida, region_city, str(year))
    os.makedirs(results_full_path, exist_ok=True)

    # Copiar clasificación a la carpeta
    for tif_file in glob.glob(os.path.join(resultados_classifier, '*.tif')):
        shutil.copy(tif_file, results_full_path)

    # Generar raster virtual .vrt con las imágenes originales
    original_images_full_path = os.path.join(raster_virtual_path, region_city)
    os.makedirs(os.path.join(root_path, original_images_full_path), exist_ok=True)

    cmd = "gdalbuildvrt {dst} {src}".format(
        dst=os.path.join(original_images_full_path, "{}.vrt".format(year)),
        src=os.path.join(imgs_orig, "*.tif"))
    print(cmd)

    process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE)
    print(process.stdout)

In [23]:
prepare_results(year=2016,
                region_city='centro/concordia',
                path_salida='data/resultados/finales',
                resultados_classifier='data/concordia2016_train_rf_t107_d1007/clasificacion_train_rf_t107_d1007',
                imgs_orig='data/imagenes/concordia_2016',
                raster_virtual_path='data/imagenes_orig')

gdalbuildvrt data/imagenes_orig/centro/concordia/2016.vrt data/imagenes/concordia_2016/*.tif
b'0...10...20...30...40...50...60...70...80...90...100 - done.\n'


In [24]:
prepare_results(year=2018,
                region_city='centro/concordia',
                path_salida='data/resultados/finales',
                resultados_classifier='data/concordia2018_train_rf_t107_d1007/clasificacion_test_rf_t107_d1007',
                imgs_orig='data/imagenes/concordia_2018',
                raster_virtual_path='data/imagenes_orig')

gdalbuildvrt data/imagenes_orig/centro/concordia/2018.vrt data/imagenes/concordia_2018/*.tif
b'0...10...20...30...40...50...60...70...80...90...100 - done.\n'
